<a href="https://colab.research.google.com/github/Talizg03/FInal-DS18/blob/main/project_26__5_25_modify1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/neheller/kits23
!cd kits23 && pip install -e .


fatal: destination path 'kits23' already exists and is not an empty directory.
Obtaining file:///content/kits23
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/deepmind/surface-distance.git to /tmp/pip-install-pj96pjr7/surface-distance_fd83916672e94cd48a9643ff7ee23a43
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/surface-distance.git /tmp/pip-install-pj96pjr7/surface-distance_fd83916672e94cd48a9643ff7ee23a43
  Resolved https://github.com/deepmind/surface-distance.git to commit 1f805cea446805e559badc80dd27f34175e4173d
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: kits23
    Found existing installation: kits23 0.1.4
    Uninstalling kits23-0.1.4:
      Successfully uninstalled kits23-0.1.4
  Running setup.py develop for kits23


In [ ]:
!kits23_download_data



Found 489 cases to download

Dowloading case_00588...: 100% 489/489 [26:20<00:00,  3.23s/it]


In [ ]:
import os

# List top-level items
print("Root:", os.listdir())

# List contents of kits23 directory
if "kits23" in os.listdir():
    print("kits23/:", os.listdir("kits23"))

# List contents of dataset
if os.path.exists("kits23/dataset"):
    print("kits23/dataset/:", os.listdir("kits23/dataset"))
else:
    print("⚠️ 'kits23/dataset/' not found.")


Root: ['.config', 'kits23', 'sample_data']
kits23/: ['kits23', '.pylintrc', 'tests', 'changelog.md', 'kits23.egg-info', 'setup.py', '.gitignore', '.git', 'LICENSE', 'pull_request_template.md', 'README.md', 'dataset']
kits23/dataset/: ['case_00091', 'case_00410', 'case_00434', 'case_00078', 'case_00587', 'case_00504', 'case_00233', 'case_00137', 'case_00219', 'case_00559', 'case_00256', 'case_00257', 'case_00292', 'case_00283', 'case_00199', 'case_00003', 'case_00564', 'case_00405', 'case_00401', 'case_00493', 'case_00520', 'case_00443', 'case_00185', 'case_00086', 'case_00243', 'case_00471', 'case_00228', 'case_00423', 'case_00445', 'case_00053', 'case_00216', 'case_00505', 'case_00262', 'case_00575', 'case_00056', 'case_00431', 'case_00586', 'case_00267', 'case_00561', 'case_00574', 'case_00501', 'case_00295', 'case_00059', 'case_00175', 'case_00403', 'case_00014', 'case_00260', 'case_00169', 'case_00120', 'case_00167', 'case_00163', 'case_00293', 'case_00400', 'case_00430', 'case_001

In [ ]:
def save_slices(case_folder, output_dir):
    import os
    import io
    import numpy as np
    import nibabel as nib
    from PIL import Image

    imaging_path = os.path.join(case_folder, "imaging.nii.gz")
    seg_path = os.path.join(case_folder, "segmentation.nii.gz")

    img = nib.load(imaging_path).get_fdata()
    seg = nib.load(seg_path).get_fdata()

    for i in range(img.shape[2]):
        slice_img = img[:, :, i]
        slice_seg = seg[:, :, i]

        # אם יש תווית 1 בפרוסה — שייך ל"normal", אחרת "abnormal"
        label = "normal" if np.any(slice_seg == 1) else "abnormal"

        # נרמול לפורמט 0-255
        slice_norm = 255 * (slice_img - np.min(slice_img)) / (np.ptp(slice_img) + 1e-5)
        slice_norm = slice_norm.astype(np.uint8)

        try:
            img_pil = Image.fromarray(slice_norm)

            # שמירה בזיכרון לבדיקה
            buf = io.BytesIO()
            img_pil.save(buf, format="PNG")
            buf.seek(0)
            test_img = Image.open(buf)
            test_img.verify()

            # רק אם תקין - לשמור לדיסק
            filename = f"{os.path.basename(case_folder)}_slice_{i:03d}.png"
            filepath = os.path.join(output_dir, label, filename)
            img_pil.save(filepath)

        except Exception as e:
            print(f"❌ Error saving slice {i} from {case_folder}: {e}")


In [ ]:
import os
import numpy as np
import nibabel as nib  # ← זה החלק שהיה חסר
from PIL import Image
from tqdm import tqdm


# 2. הגדרת הנתיבים:

In [ ]:
DATASET_DIR = 'kits23/dataset'  # הנתיב לתיקיית הדאטה
OUTPUT_DIR = 'slices_output'

os.makedirs(os.path.join(OUTPUT_DIR, "normal"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "abnormal"), exist_ok=True)


# 3. הגדרת פונקציית save_slices:



In [ ]:
def save_slices(case_folder, output_dir):
    imaging_path = os.path.join(case_folder, "imaging.nii.gz")
    seg_path = os.path.join(case_folder, "segmentation.nii.gz")
    img = nib.load(imaging_path).get_fdata()
    seg = nib.load(seg_path).get_fdata()

    for i in range(img.shape[2]):
        slice_img = img[:, :, i]
        slice_seg = seg[:, :, i]

        # תווית: אם יש 1 => נורמל, אחרת לא נורמל
        label = "normal" if np.any(slice_seg == 1) else "abnormal"

        # נרמול התמונה
        slice_norm = 255 * (slice_img - np.min(slice_img)) / (np.ptp(slice_img) + 1e-5)
        slice_norm = slice_norm.astype(np.uint8)

        try:
            img_pil = Image.fromarray(slice_norm)

            # בדיקת תקינות של התמונה
            buf = io.BytesIO()
            img_pil.save(buf, format="PNG")
            buf.seek(0)
            test_img = Image.open(buf)
            test_img.verify()

            # שמירה לתיקייה המתאימה
            filename = f"{os.path.basename(case_folder)}_slice_{i:03d}.png"
            filepath = os.path.join(output_dir, label, filename)
            img_pil.save(filepath)

        except Exception as e:
            print(f"❌ Error saving slice {i} from {case_folder}: {e}")


#4. סריקת הקייסים והפקת הפרוסות

In [ ]:
import os
from tqdm import tqdm
import io  # ✅ Ensure this is included if you're using io.BytesIO() etc.

cases = sorted([
    os.path.join(DATASET_DIR, c)
    for c in os.listdir(DATASET_DIR)
    if c.startswith("case_")
])

print("Saving slices into normal / abnormal folders...")
for case_folder in tqdm(cases):
    try:
        save_slices(case_folder, OUTPUT_DIR)
    except FileNotFoundError as e:
        print(f"⚠️ Skipping {case_folder}: File not found - {e}")
    except Exception as e:
        print(f"❌ Error saving slices from {case_folder}: {e}")

print("✅ Done saving slices.")



Saving slices into normal / abnormal folders...


100%|██████████| 489/489 [3:13:08<00:00, 23.70s/it]

✅ Done saving slices.


In [ ]:
import shutil

# Replace 'slices_output' with your actual folder name if different
shutil.make_archive('slices_output', 'zip', 'slices_output')


'/content/slices_output.zip'

In [ ]:
from google.colab import files
files.download('slices_output.zip')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#5. (אופציונלי) בדיקה כמה תמונות נשמרו בכל קטגוריה

In [ ]:
normal_count = len(os.listdir(os.path.join(OUTPUT_DIR, "normal")))
abnormal_count = len(os.listdir(os.path.join(OUTPUT_DIR, "abnormal")))

print(f"✅ Total slices saved:")
print(f"Normal: {normal_count}")
print(f"Abnormal: {abnormal_count}")


✅ Total slices saved:
Normal: 93842
Abnormal: 157069


#6. דחיסת הקבצים ל-ZIP (כדי לשמור למחשב)

In [ ]:
import shutil

shutil.make_archive("slices_output", "zip", OUTPUT_DIR)


'/content/slices_output.zip'

#Set Up the Data Pipeline

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128, 128)  # or (256, 256) depending on your slice size
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    'slices_output',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'slices_output',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)


Found 200730 images belonging to 2 classes.
Found 50181 images belonging to 2 classes.


#Define the CNN Model

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(*IMG_SIZE, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,569 (396.75 KB)

 Trainable params: 101,569 (396.75 KB)

 Non-trainable params: 0 (0.00 B)

#. Train the Model

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6273/6273 ━━━━━━━━━━━━━━━━━━━━ 6646s 1s/step - accuracy: 0.7512 - loss: 0.4520 - val_accuracy: 0.8625 - val_loss: 0.3145
Epoch 2/10
6273/6273 ━━━━━━━━━━━━━━━━━━━━ 6458s 1s/step - accuracy: 0.8757 - loss: 0.2852 - val_accuracy: 0.8808 - val_loss: 0.2782
Epoch 3/10
1369/6273 ━━━━━━━━━━━━━━━━━━━━ 1:17:21 947ms/step - accuracy: 0.8930 - loss: 0.2483

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp slices_output.zip /content/drive/MyDrive/


In [ ]:
# Get final accuracy and validation accuracy from training history
train_acc = history.history['accuracy'][-1] * 100
val_acc = history.history['val_accuracy'][-1] * 100

print(f"✅ Final Training Accuracy: {train_acc:.2f}%")
print(f"✅ Final Validation Accuracy: {val_acc:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
model.evaluate(val_gen)

# Save the trained model
model.save("normal_abnormal_classifier.h5")


# Test the Model on a New Case

In [ ]:
import nibabel as nib
import numpy as np
from PIL import Image
import os

# Path to a test case
test_case_folder = "kits23/dataset/case_00500"

# Load the volume
img_path = os.path.join(test_case_folder, "imaging.nii.gz")
img = nib.load(img_path).get_fdata()

# Choose a slice index to test (e.g., slice #50)
i = 50
slice_img = img[:, :, i]

# Normalize to 0–255 and convert to PIL image
slice_norm = 255 * (slice_img - np.min(slice_img)) / (np.ptp(slice_img) + 1e-5)
slice_norm = slice_norm.astype(np.uint8)

img_pil = Image.fromarray(slice_norm).resize((128, 128)).convert("RGB")

# Convert to array and scale 0–1
img_arr = np.array(img_pil) / 255.0
img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension


#Predict Using the Model

In [ ]:
pred = model.predict(img_arr)[0][0]

# Threshold at 0.5 to classify
label = "normal" if pred >= 0.5 else "abnormal"
confidence = pred if pred >= 0.5 else 1 - pred

print(f"🧠 Prediction: {label} ({confidence * 100:.2f}%)")

